In [1]:
from Bio import AlignIO, SeqIO, Seq, pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
import os
import numpy as np

/home/norbert/miniconda3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
min_length_aligned_sequence = 30 #Minimal lenght of sequence which could be an exon
extreme_homology = 0.97 #percentage of homology of sequence, treshold #I assume two faulty aligned nucleotides per 100 (98%) and one more nt because sometimes latest nt can move from end of one sequence to beginning next sequence
middle_homology = 0.9 #used for classification of internal exons

In [3]:
#MEGA ISTOTNE PYTANIE - CO WYŚWIETLAMY????
                    #CZY W KOLUMNIE SEQUENCE WYSWIETLAMY SEKWENCJE TRANKRYPTU CZY GENU?
#######################################################################################
#######################################################################################


#cel na chwile potem:
    #wziac pliki wyslane przez pawla i w bio pythonie je polaczyc. Jeden plik fasta zawiera po trzy scaffoldy dla kazdego gatunku oraz trzy odczyty trinity. Biopython musi je wziac ze soba i polaczyc wywalajac gapy.

#pliki adama sa inaczej skonstruowane i są bardziej zawiłe: jeden plik fasta zawiera po trzy gatunki, hiemalis longa i gracilis. Kazdy z gatunkow ma tam zapisany scaffold i trinity. Olejmy to na razie i filtrowanie ich zrobimy potem
            #chce teraz zrobic ta tabelke ktora bedzie dawala sprawnie pozycje w surowym pliku
#moje pliki bedace inputem (te po maffcie) maja opisane jaki to organizm i jaki transrypt np EG numerki 308. Taki sam schemat jest w plikach surowych. znajdziemy czesc wspolna np. jako pierwsze x liter



In [17]:
def cutting_scrap(path_to_file_after_MAFFT, path_to_file_before_MAFFT, acceptable_gap_length):
    data_frames = []
    gaps_signs = "-" * acceptable_gap_length #maximum length of gaps in sequence in one exon's sequence
    
    def extracting_strands_from_alignment(alignment):
        seq1 = alignment[0].seq #transcript sequence
        seq2 = alignment[1].seq #gene sequence
        return seq1, seq2

    #def setting_first_nucleotide_pair(alignment): #ta funkcja ucina nam 6 odczytow #to nam nie działa więc nie brnę dalej. Nie działa prawdopodobnie dlatego, że jest ona próba odwzorowania kodu, który spokojnie sobie działał w pętli while true
        #if "-" in alignment[:, 0]: #deleting mismatches at the first nucleotide
            #return alignment[:, 1:]
        #else:
            #return alignment
            #print(f"alignment after cut:\n{alignment}") #chekpoint


    #def setting_last_nuckleotide_pair(alignment): #to nam nie działa więc nie brnę dalej
        #if "-" in alignment[:, -1]: #deleting mismatches at the end
            #alignment = alignment[:, :-1]
            #return alignment 
        #else:
            #return alignment #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
                    

    def finding_gap_index(sequence, acceptable_gap_length):
        return sequence.find("-" * acceptable_gap_length) 


    def reversed_finding_gap_index(sequence, acceptable_gap_length):
        return sequence.rfind("-" * acceptable_gap_length) + acceptable_gap_length #now we have to add length of gap to index, because python starts counting from 0 nt, and we just change direction of counting.

    def setting_distance_between_first_nt_and_gap(is_it_exon):
        if is_it_exon == True:
            if first_gap_position_seq2 < 0: #when in second strand there are no gaps - it means, second strand is correctly prepared
                distance_between_first_nt_and_gap = first_gap_position_seq1
            else:
                distance_between_first_nt_and_gap = min(first_gap_position_seq1, first_gap_position_seq2)
            return distance_between_first_nt_and_gap
        elif is_it_exon == False:
            if last_gap_position_seq2 < 0: #it means, if there is no gaps in second sequention, gap position is gap position in seq1. Additional, rfind() function return "-1" if it won't find any gaps in seq
                distance_between_first_nt_and_gap = (len(seq1) - last_gap_position_seq1) -1
                #print("pierwszy", distance_between_first_nt_and_gap) #checkpoint
            else:
                distance_between_first_nt_and_gap = (len(seq1) - max(last_gap_position_seq1, last_gap_position_seq2))
            return distance_between_first_nt_and_gap
                #print("drugi", distance_between_first_nt_and_gap) #checkpoint

    
    def save_to_file(final_data_frame, filename):
        if os.path.isfile(filename):
            user_input = input("File already exists. Do you want overwrite? y/n: ")
            if user_input == "n":
                base_name, ext = os.path.splitext(filename)
                i = 1
                while os.path.isfile(f"{base_name}_{i}{ext}"):
                    i += 1
                filename = f"{base_name}_{i}{ext}"
                final_data_frame.to_csv(filename, sep = "\t")
                
            elif user_input == "y":
                final_data_frame.to_csv(filename, sep = "\t")
            else:
                print("Clarify your answer. Nothing has done.")
                
        else:
            final_data_frame.to_csv(filename, sep = "\t")
            
    
    for filename in os.listdir(path_to_file_after_MAFFT):
        file = os.path.join(path_to_file_after_MAFFT, filename)
        if not os.path.isfile(file):
            continue
        if file.endswith(".aln"):
            alignment = AlignIO.read(file, "clustal")



            
            
        #propozycje rozwiazania problemu:
            #tu musi byc zawarta funkcja parujaca sie z zadanym plikiem alignment
            #bierzemy nazwe pliku ktory jest obecnie zaladowany do pythona i przeszukujemy tym stringiem folder ze sciezki podanej w poszukiwaniu nazwy pliku, ktora wyglada tak: EL_OG0030308.fasta. Potem to zmienimy
            #print(filename) jest tożsame z alignment[0].id, ale ma dodatkowe rozszerzenie .aln ktore nie stanowi problemu. Zatem obie te wersje sa zalezne od siebie, wiec potem bedzie to gorszy wybor z racji tego ze nazewnictwo zalezy ode mnie nie od programu. Chyba ze tak skontruuje program, ze on bedzie nazywal te pliki outputowe adama tak jak ja chce, wedlug mojego pomyslu nazewnictwa!
            #print(alignment[1].id) #odrzucamy wyszukiwanie po id drugiej nici bo ona raz sie nazywa scaffold a raz backbone. 


            #czy ten fragment trzeba zamknąć do funkcji i wyeksportować ją na początek - mówie tu o kwestiach estetycznych
            for filename_before_MAFFT in os.listdir(path_to_file_before_MAFFT):
                file_before_MAFFT = os.path.join(path_to_file_before_MAFFT, filename_before_MAFFT)
                if not os.path.isfile(file_before_MAFFT):
                    #print(f"This thingy: {str(filename_before_MAFFT)} could not be used to pair with alignment. Check it or ignore")
                    continue
                if filename_before_MAFFT.endswith(".fasta"):
                    #print(filename_before_MAFFT)
                    if filename.startswith(filename_before_MAFFT[:12]):
                        #print(f"filename: {filename}")
                        max_len = 0
                        max_genomic_seq = ""
                        #przypisujemy tutaj sekwencję genomową (dłuższą) do zmiennej na której bedziemy dalej operować
                        for record in SeqIO.parse(file_before_MAFFT, "fasta"):
                            sequence = record.seq
                            if len(sequence) > max_len:
                                max_len = len(sequence)
                                max_genomic_seq = record.seq   
                         
            while True:
                is_it_exon = True
                #alignment = setting_first_nucleotide_pair(alignment)
                first_nucleotides_pair = alignment[:, 0]#first nucleotides in both strands as variable
                if "-" in first_nucleotides_pair: #deleting mismatches at the start 
                    alignment = alignment[:, 1:]
                    #print(f"alignment after cut:\n{alignment}") #chekpoint
                    continue
    
                
                seq1, seq2 = extracting_strands_from_alignment(alignment)

                
                first_gap_position_seq1 = finding_gap_index(seq1, acceptable_gap_length)
                first_gap_position_seq2 = finding_gap_index(seq2, acceptable_gap_length)
                #print(f"first gap position in first sequence:{first_gap_position_seq1}, in 2nd sequence: {first_gap_position_seq2}") #checkpoint

                distance_between_first_nt_and_gap = setting_distance_between_first_nt_and_gap(is_it_exon)
                
                local_alignment = pairwise2.align.localxx(seq1[:distance_between_first_nt_and_gap], seq2[:distance_between_first_nt_and_gap], one_alignment_only = True) #"xx"  means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment to count homology
                #print("local_alignment:", local_alignment) #checkpoint
                   
                try: 
                    local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short, sequence has not exons or other fault. \n")
                        break
                    continue
                    
                
                if local_homology_percentage <= extreme_homology:
                    #print(f"\n Too low score of homology. \n alignment: {alignment[:, :50]} \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, percent: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                    alignment = alignment[:, distance_between_first_nt_and_gap:] 
                    
                else:
                    if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                        #print(f"\n alignment is too short. \n distance between sequences {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                        alignment = alignment[:, distance_between_first_nt_and_gap:]
                    else:
                        #print(f"Sequence {alignment[0].id} cut properly from LEFT side, \n distance between sequences: {distance_between_first_nt_and_gap}"), alignment: {alignment[:, :50]},
                        break
    
               
        
            
        ######################### TERAZ OD PRAWEJ DO LEWEJ #######################  

    
            
            while True:
                is_it_exon = False
                last_nucleotides_pair = alignment[:, -1]#last nucleotides in both strands as variable
                if "-" in last_nucleotides_pair: #deleting mismatches at the end
                    alignment = alignment[:, :-1]
                     #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
                    continue
                             
                seq1, seq2 = extracting_strands_from_alignment(alignment)

                last_gap_position_seq1 = reversed_finding_gap_index(seq1, acceptable_gap_length)
                last_gap_position_seq2 = reversed_finding_gap_index(seq2, acceptable_gap_length)
                #print("seq1:", last_gap_position_seq1, "seq2:", last_gap_position_seq2) #checkpoint
                
                distance_between_first_nt_and_gap = setting_distance_between_first_nt_and_gap(is_it_exon)
                #print("drugi", distance_between_first_nt_and_gap)
                        
                local_alignment = pairwise2.align.localxx(seq1[-distance_between_first_nt_and_gap:], seq2[-distance_between_first_nt_and_gap:], one_alignment_only = True) #"xx" means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment
                #print("local_alignment:", local_alignment) #checkpoint
                
                
        
                try:
                    if distance_between_first_nt_and_gap != 0:
                        local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
                    else:
                        break
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                        break
                    continue
        
                
                if local_homology_percentage <= extreme_homology:
                     #print(f"\n Too low homology! \n alignment: {alignment[:, -50:]} \n score: {local_alignment[0].score}, \distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                    alignment = alignment[:, :-distance_between_first_nt_and_gap] 
                    
                else:
                    if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                        #print(f"\n Alignment is too short!. \n distance: {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                        alignment = alignment[:, :-distance_between_first_nt_and_gap]
                    else:
                        #print(f"Sequence: {alignment[0].id} cut properly from RIGHT side, alignment: \n {alignment[:, -50:]}")
                        break

        
        
            
            nucleotides = ["a", "t", "g", "c", "A", "T", "G", "C"]
            index = 0
            
            id = []
            exon_or_intron_number = []
            is_it_intron = []
            class_of_exon = []
            percent_of_homology_for_exons = []
            length = []
            first_nt_position = []
            last_nt_position = []
            first_10_nt = []
            last_10_nt = []
            path = []
            sequence = []

            
            while True:
                try: 
                    if distance_between_first_nt_and_gap != 0:
                        local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) #that sign "[]" becouse local_alignment's type is list
                    else:
                        break
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                        break
                    continue
                
                if len(seq1) > 1 and seq1[0] != gaps_signs:#mining data from exons
                    index += 1
                    last_nt_exon_index = seq1.find(gaps_signs) 
                
                    if last_nt_exon_index == -1: #it means if it is last exon, if find() don't find any gap signs at the end
                        last_nt_exon_index = len(seq1)
                                
                    alignment_of_exon = pairwise2.align.localxx(seq1[:last_nt_exon_index], seq2[:last_nt_exon_index], one_alignment_only = True)
                        #print( "\n alignment since start to the end of exon:", alignment_of_exon[0])
                    percent_of_homology = alignment_of_exon[0].score / last_nt_exon_index
                        #print("exon analising finished")
                            
            
                    #extending lists to make bigger dictionary and then data frame
                    id.append(alignment[0].id)
                    exon_or_intron_number.append(f"E{index}")
                    is_it_intron.append(False)
                    percent_of_homology_for_exons.append(round(percent_of_homology * 100, 2))
                    length.append(last_nt_exon_index)
                    first_10_nt.append(str(seq1[:10]))
                    last_10_nt.append(str(seq2[last_nt_exon_index-10 : last_nt_exon_index])) 
                    sequence.append(str(seq2[:last_nt_exon_index]).replace("-", ""))
                    path.append(str(file))


                    #adding indices of start and end of exons
                    first_nt_index_of_exon = max_genomic_seq.find(str(seq2[:last_nt_exon_index]).replace("-", ""))
                    if first_nt_index_of_exon == -1:
                        first_nt_position.append(0)
                        print(f"That exon: {str(seq2[:last_nt_exon_index]).replace('-', '')} comes from genomic sequence and cannot be found in file_before_MAFFT.")

                    else:
                        first_nt_position.append(first_nt_index_of_exon+1)

                    if first_nt_index_of_exon == -1:
                        last_nt_position.append(0)
                    else:
                        last_nt_position.append(first_nt_index_of_exon + len(seq2[:last_nt_exon_index]))
                    
                    #describing class of exon
                    if percent_of_homology >= 0.9 and last_nt_exon_index >= min_length_aligned_sequence :
                        class_of_exon.append("1")
                    elif percent_of_homology < 0.9 and percent_of_homology > 0 and last_nt_exon_index < min_length_aligned_sequence:
                        class_of_exon.append("2")
                    else:
                        class_of_exon.append("3")
            
                    seq1 = seq1[last_nt_exon_index:]
                    seq2 = seq2[last_nt_exon_index:]
                    #print("analising exon finished") #checkpoint
                            
                #expanding index
                if len(seq1) > 1 and seq1[0] == "-":
                    for i, nukleotide in enumerate(seq1):
                        if nukleotide not in nucleotides:
                            last_nt_intron_index = i+1
                        else:
                            break

                    #extending lists to make bigger dictionary and then data frame
                    id.append(alignment[0].id)
                    exon_or_intron_number.append(f"I{index}")
                    is_it_intron.append(True)
                    class_of_exon.append(None)
                    percent_of_homology_for_exons.append(None)
                    length.append(last_nt_intron_index)
                    #first_nt_position.append(0)
                    #last_nt_position.append(0)
                    first_10_nt.append(str(seq2[:10]))
                    last_10_nt.append(str(seq2[last_nt_intron_index-10 : last_nt_intron_index]))
                    sequence.append(str(seq2[:last_nt_intron_index]).replace("-", ""))
                    path.append(str(file))

                    
                    first_nt_index_of_intron = max_genomic_seq.find(str(seq2[:last_nt_intron_index]))
                    if first_nt_index_of_intron == -1:
                        first_nt_position.append(0)
                    else:
                        #print(first_nt_index_of_intron)
                        first_nt_position.append(first_nt_index_of_intron+1)

                    if first_nt_index_of_intron == -1:
                        last_nt_position.append(0)
                    else:
                        last_nt_position.append(first_nt_index_of_intron + len(seq2[:last_nt_intron_index]))

                    #tutaj damy liste ktorej inputem bedzie ostatni element listy z exonow, do ktorego dodamy dlugosc tego eksonu +1 zeby znalezc poczatkowy index intronu
                    seq1 = seq1[last_nt_intron_index:]
                    seq2 = seq2[last_nt_intron_index:]
                    
                else:
                     break
                        
            dictionary = {"ID":id,
                          "exon_or_intron_number":exon_or_intron_number, 
                          "is_it_intron":is_it_intron, 
                          "class_of_exon":class_of_exon, 
                          "percent_of_homology_for_exons":percent_of_homology_for_exons, 
                          "length":length,
                          "first_nt_position" : first_nt_position,
                          "last_nt_position" : last_nt_position,
                          "first_10_nt" : first_10_nt,
                          "last_10_nt" : last_10_nt, 
                          "path":path, 
                          "sequence" : sequence}
            data_frame = pd.DataFrame(dictionary)
            #print(data_frame)
        data_frames.append(data_frame)
        #print(data_frames)


    final_data_frame = pd.concat(data_frames, ignore_index=True)
    final_data_frame.index = np.arange(1, len(final_data_frame) + 1)
    print(final_data_frame)
    
    save_to_file(final_data_frame, "przyciete_introny.tsv")


cutting_scrap('/home/norbert/mrdn/euglena/kod_i_pliki/alingment_gen_transkrypt/', "/home/norbert/mrdn/euglena/kod_i_pliki/surowe_pliki_plus_minus_500", 2)






 
 EL_OG0030308_t2_g2_EL_PAM1_GP153_OV05 sequence is too short, sequence has not exons or other fault. 

                                        ID exon_or_intron_number is_it_intron  \
1    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    E1          0.0   
2    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    I1          1.0   
3    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    E2          0.0   
4    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    I2          1.0   
5    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    E3          0.0   
..                                     ...                   ...          ...   
144  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    E6          0.0   
145  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    I6          1.0   
146  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    E7          0.0   
147  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                    I7          1.0   
148

File already exists. Do you want overwrite? y/n:  y
